In [1]:
# import warnings
# warnings.simplefilter('ignore')
# warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')


import spacy
# gpu = spacy.prefer_gpu()
# print('GPU:', gpu)
# pip install -U spacy[cuda100]
# python -m spacy validate
# python -m spacy download en_core_web_sm

from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint

nlp = en_core_web_sm.load()
import re
from stopwords import *
import nltk
from preprocess import *
# from feature import *

from textblob import TextBlob
import collections

# from spacy.symbols import cop, acomp, amod, conj, neg, nn, nsubj, dobj,prep,advmod
# from spacy.symbols import VERB, NOUN, PROPN, ADJ, ADV, AUX, PART

pattern_counter = collections.Counter()

# from nltk.corpus import stopwords
# import networkx as nx
# from MongoDB import MongoDB
import os
import pickle
from eda import *
from glob import glob
from product import *

斷詞辭典 已取得
negative-words.txt
positive-words.txt
total-words 已取得


# 斷詞辭典

In [2]:
len(stopwords)

2562

In [3]:
opinion_lexicon.keys()

dict_keys(['negative-words', 'positive-words', 'total-words'])

# pickle 提取

In [4]:
# _,category1,category2,_ = DVD_Player().getAttr()
# _,category1,category2,_ = Cameras().getAttr()
_,category1,category2,_ = Cell_Phones().getAttr()
# _,category1,category2,_ = GPS().getAttr()
# _,category1,category2,_ = Keyboards().getAttr()

glob('data/new_%s_%s.pickle'%(category1,category2))

['data/new_Cell Phones & Accessories_Cell Phones.pickle']

In [5]:
# reload a file to a variable   
with open(glob('data/new_%s_%s.pickle'%(category1,category2))[0], 'rb') as file:
    print(file.name)
    PROD_DICT = pickle.load(file)
    category1,category2 = file.name.replace(".pickle","").split("_")[1:]
    


data/new_Cell Phones & Accessories_Cell Phones.pickle


# 提取所有規格關鍵字

In [6]:
import warnings
from tqdm import tqdm
from collections import Counter,OrderedDict

feature_counter1 = Counter()
feature_counter2 = Counter()
feature_counter3 = Counter()
warnings.filterwarnings('ignore')

import collections
freq_words = collections.Counter()
feat_words = collections.Counter()
opinion_words = collections.Counter()
cooccurs_words = collections.Counter()

_Sents = []
keywords_list = []
# 其中 freq_words 是單字出現的頻率 , _Sents 是文章中所有的句子

i = 0

# total_keywords = []
Products = len(PROD_DICT.items())
print("%s Products\n"%(Products))    

if not os.path.exists('View'):
    os.makedirs('View')
  
Reviews = 0
fn = 'View/%s_%s_keywords.txt'%(category1,category2)
if not os.path.exists(fn):
    print("Start get Product keywords...")
    with open('View/Prod_keyword.txt',"w",encoding="utf-8") as f:
        # with open("Prod_keyword.txt","w",encoding="utf-8") as f:
        with tqdm(total=Products) as pbar:
            for asin,DATA_DICT in PROD_DICT.items():
                REVIEW_ITEM_LIST = DATA_DICT['REVIEW_ITEM_LIST']
                title = DATA_DICT['title']
                description = DATA_DICT['description']
                feature = DATA_DICT['feature']      
                dash_keywords,noun_keywords,newkeywords, newdescription = getKeywordFeat(description,feature)
#                 print(noun_keywords)
                f.write("* asin:" + asin+"\n")
                if "(new Date()).getTime();" in title : f.write("title:" + asin+"\n")
                else: f.write("title:" + title+"\n")
                f.write('dash_keywords:\n' + str(dash_keywords)+"\n");f.write('noun_keywords2:\n' + str(noun_keywords)+"\n")

                f.write('* keywords:\n' + str(newkeywords)+"\n")
#                 f.write('* newdescription:\n' + str(newdescription)+"\n")
#                 f.write('* pos_newdescription:\n' + str(get_pos_sequence(newdescription)[1])+"\n")
                f.write('************************************************************************'+"\n")
                f.write('************************************************************************'+"\n")      
                for pair in REVIEW_ITEM_LIST:
                    review_ID = pair["review_ID"] 
                    review = pair["review"]  
                    lemm_review = pair["lemm_review"] 
                    vote = pair["vote"] 
                    summary = pair["summary"] 
                    pbar.set_description("Product : %s , Processing ID %s" % (asin,review_ID))

                    dash_keywords2,noun_keywords2,newkeywords2, _ = getKeywordFeat_2(lemm_review)                    
                    dash_keywords3,noun_keywords3,newkeywords3, _ = getKeywordFeat_2(summary)
                    feature_counter2.update(newkeywords2)
                    feature_counter3.update(newkeywords3)
                    Reviews += 1 
                    
                
                feature_counter1.update(newkeywords)
                
                pbar.update(1)
                
                
    important_features = OrderedDict(sorted(feature_counter1.items(), key=lambda pair: pair[1], reverse=True))
    important_features = [(word,important_features[word]) for word in important_features if important_features[word]>5]
    print("Count : %s"%(len(important_features)))
    
    important_features2 = OrderedDict(sorted(feature_counter2.items(), key=lambda pair: pair[1], reverse=True))
    important_features2 = [(word,important_features2[word]) for word in important_features2 if important_features2[word]>5]
    print("Count : %s"%(len(important_features2)))
    
    important_features3 = OrderedDict(sorted(feature_counter3.items(), key=lambda pair: pair[1], reverse=True))
    important_features3 = [(word,important_features3[word]) for word in important_features3 if important_features3[word]>5]
    print("Count : %s"%(len(important_features3)))

    fn = 'View/%s_%s_keywords.txt'%(category1,category2)
    with open(fn,"w",encoding="utf-8") as f :
        total_keywords = set()
        for word , v in important_features:        
            f.write("%s:%s \n"%(word,v))  
            if v > 20 : total_keywords.add(word)  
            
    fn2 = 'View/%s_%s_keywords2.txt'%(category1,category2)    
    with open(fn2,'w',encoding="utf-8") as f:
        total_keywords2 = set()
        for word , v in important_features2:        
            f.write("%s:%s \n"%(word,v))   
            total_keywords2.add(word)  
            
    fn3 = 'View/%s_%s_keywords3.txt'%(category1,category2)
    with open(fn3,'w',encoding="utf-8") as f:
        total_keywords3 = set()
        for word , v in important_features3:        
            f.write("%s:%s \n"%(word,v))   
            total_keywords3.add(word)                   
    total_keywords = total_keywords2
else:
    fn = 'View/%s_%s_keywords2.txt'%(category1,category2)
    print('load %s keywords...'%(fn))
    total_keywords = set()
    with open(fn,'r',encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            k,v = line.split(":")
            total_keywords.add(k)
            
# pbar.close()
    # total_keywords = set(total_keywords) # 檢查重要程度 篩選字典裡的單字
    # open("total_keywords.txt",'w',encoding="utf-8").write(str(total_keywords))

  0%|          | 0/107 [00:00<?, ?it/s]

107 Products

Start get Product keywords...


Product : B01GW45SB8 , Processing ID 1485043200: 100%|██████████| 107/107 [06:31<00:00,  3.66s/it]

Count : 166
Count : 787
Count : 59


# TF-IDF 關鍵字濾除 (each newkeywords)

In [7]:
# import os
# import sys
# from sklearn import feature_extraction
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import CountVectorizer

# # def Importance(document, sorted_words):
# #     scores = {}
# #     sent = get_token_sent(document)
# #     for word in sent:
# #         try:
# #             scores[word] = dict(sorted_words)[word]
# #         except Exception as e:
# #             pass
# #     scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
# #     return scores[:20]

# vectorizer = CountVectorizer()  # 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
# transformer = TfidfTransformer()  # 该类会统计每个词语的tf-idf权值
# tfidf = transformer.fit_transform(
#     vectorizer.fit_transform(keywords_list))  # 第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
# word = vectorizer.get_feature_names()  # 获取词袋模型中的所有词语
# weight = tfidf.toarray()  # 将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

# print("Calculate TF-IDF finished")

# scores = {}
# for i in range(len(weight)):
#     # print (u"-------这里输出第",i,u"类文本的词语tf-idf权重------")
#     for j in range(len(word)):
#         # print(word[j],weight[i][j])
#         scores[word[j]] = weight[i][j]


# # fout = 'TF-IDF/%s_%s.txt'%(big_categories,small_categories)
# if not os.path.isdir("TF-IDF"):
#     os.mkdir("TF-IDF")
    
# with open('TF-IDF/%s_%s.txt'%(category1,category2),'w',encoding='utf-8') as f:   
#     f.write("word" + '\t' + "tf-idf-weight" + '\n')
#     for k, v in scores.items():
#         if float(v) <= 0: continue
# #         print(k,v)
#         f.write(str(k) + '\t' + str(v) + '\n')

# # sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
# sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
# sorted_words = [word[0] for word in sorted_words]
# print("\nTop 20 Important words", sorted_words[:20])

# # total_keywords = set(sorted_words)

# 自然語言處理 -- Pointwise Mutual Information

In [8]:
# 使用total_keywords計算pmi
from math import log
# need pmi feature words & opinions
def p(x): # p(x) 計算單字 x 出現的機率
    return freq_words[x]/float(len(freq_words))

def pxy(x,y): # pxy(x,y) 計算單字 x 和單字 y 出現在同一個句子的機率
#     print((lambda s :  x in s and y in s ,_Sents))
#     filter()函數用於過濾序列，過濾掉不符合條件的元素
    return (len(list(filter(lambda s :  x in s and y in s ,_Sents)))+1)/ float(len(_Sents) )

def pmi(x,y): # pmi(x,y) 計算單字 x 和單字 y 的 Pointwise Mutual Information
    try:
#         print("X :%s Y: %s"%(x,y))
#         print('pxy(x,y)',pxy(x,y))
#         print('p(x)',p(x))
#         print('p(y)',p(y))
#         print('log(pxy(x,y)/(p(x)*p(y)),2)',log(pxy(x,y)/(p(x)*p(y)),2))
        return  log(pxy(x,y)/(p(x)*p(y)),2) 
    except Exception as e :
#         print(e)
        return 0
    
def pmi_fopair(sentence):
    doc = nlp(sentence)
    noun_,op_ = [],[]
    cand_pairs = []
    cand_pairs_score = []
    for token in doc:
        if token.pos in [ADJ,ADV,VERB]: op_.append(token.text)
        if token.pos in [NOUN]: noun_.append(token.text)

    for f in noun_:
        for o in op_:
            pair = (f,o)
            score = pmi(f,o)
#             if score > 0: cand_pairs.append((pair,score))
            if score > 0: cand_pairs.append(pair)
            cand_pairs_score.append((pair,score))

#             cand_pairs.append((pair,score))
#     print(sentence)
#     print(cand_pairs)
    return cand_pairs,cand_pairs_score

In [9]:
# _Sents[3]

# p('year')
# p('close')

# pmi_fopair("player close year absolutly amazing piece equipment")

# 計算FO-PAIR以及句子抽取

In [15]:
import pandas as pd
i = 0    

if not os.path.exists('Train'):
    os.makedirs('Train')
    
pattern_text = open("View/review_pattern.txt",'w',encoding="utf-8")
# training_data = open("View/%s_%s_train.txt"%(category1,category2),'w',encoding="utf-8")
# training_data.write("rev_sent | polarity,subjectivity | f1_o1 f2_o2 f3_o3 | summary" + '\n') 

topic_train_df = {}
key_train_df = {}
i = 0
j = 0
with tqdm(total=Reviews) as pbar:
    with open("View/result.txt",'w',encoding="utf-8") as result:
        for asin,DATA_DICT in PROD_DICT.items():
            REVIEW_ITEM_LIST = DATA_DICT['REVIEW_ITEM_LIST']
            title = DATA_DICT['title']
#             if "(new Date()).getTime();" in title : continue
            description = DATA_DICT['description']
            feature = DATA_DICT['feature']
            big_categories = DATA_DICT["category1"]
            small_categories = DATA_DICT["category2"]
            if "(new Date()).getTime();" not in title :result.write("title:" + title+"\n")
            else: result.write("title:" + asin+"\n")

            result.write('************************************************************************'+"\n")      
            for pair in REVIEW_ITEM_LIST:
                review = pair["review"]  
                rev_dash_keywords, newreview = cleanReview(review)
#                 f.write('newreview:\n'+newreview+"\n")
                vote = pair["vote"]  
                overall = pair["overall"] 
                summary = pair["summary"]
                lemm_review = pair["lemm_review"]
                lemm_summary = pair["lemm_summary"]
                lemm_review_len = pair['lemm_review_len'] 
                lemm_summary_len = pair['lemm_summary_len'] 

                review_ID = pair["review_ID"]
                
                
                pbar.set_description("Product : %s , Processing ID %s  " % (asin,review_ID))
                pbar.update(1)
            
                summ_token_set = set(nltk.word_tokenize(lemm_summary))
                if len(summ_token_set & total_keywords) == 0: continue
                result.write('overall:' + str(overall) + "\n")
                result.write('review:\n' + review + "\n")
                result.write('summary:\n' + summary + "\n")
                result.write("************************************************\n")
     
                extract_sents = sentence_extract_blob(review)
                keyword_list = []
                rev_summ_row = {}
#                 lemm_review,_ = lemm_sent_process2(review, remove_stopwords=False, summary=False, mode="spacy", withdot=True)
#                 print('lemm_review:')
#                 print(lemm_review)
                extract_sents = sentence_extract_blob(lemm_review)
                for rev_sent in extract_sents:
                    fop_row = {} # fop train row
                    fop_row['overall'] = overall
                    sentiment = TextBlob(rev_sent).sentences[0].sentiment
                    lemm_sent = rev_sent
#                     rev_sent = rev_sent.replace("s.","")
#                     rev_sent = re.sub(r'(?:^| )\w(?:$| )', " ", rev_sent).strip() # remove single alphbet
#                     rev_sent = rev_sent.replace(" ve "," ").replace(" ha "," ")                  
#                     lemm_sent,pos_sent = lemm_sent_process2(rev_sent, remove_stopwords=False, summary=False, mode="spacy", withdot=False)
    #                     _,pos_sent = get_pos_sequence(rev_sent,remove_stopwords=False,summary=False,mode = "spacy",withdot =True)
#                     if len(pos_sent) == 0: continue
#                     print('lemm_sent:',lemm_sent)
                    token_set = set(nltk.word_tokenize(rev_sent))
                    mention_features = list(token_set & total_keywords)                   
                    POS_fops = FO_rule_POS(lemm_sent).run()
                    DEP_fops = FOP_rule_Depend(lemm_sent).run()
                    POS_fops = [(f,o) for f,o in POS_fops if f in total_keywords]
                    DEP_fops = [(f,o) for f,o in DEP_fops if f in total_keywords]
#                     PMI_fops,cand_pairs_score = pmi_fopair(lemm_sent) # 皆為空有問題                     

                    result.write("************************************************\n")                     
                    result.write('rev_sent:\n' + rev_sent + "\n") ;fop_row['rev_sent'] = rev_sent
                    result.write('lemm_sent:\n' + lemm_sent) ;fop_row['lemm_sent'] = lemm_sent
                    
#                     result.write(str(pos_sent)+"\n");pattern_text.write(str(pos_sent)+"\n") ; 
                    result.write('mention features:' + str(mention_features) + "\n")
                    result.write("polarity: %s subjectivity: %s \n"%(sentiment.polarity,sentiment.subjectivity))
                    fop_row['polarity'] = sentiment.polarity;fop_row['subjectivity'] = sentiment.subjectivity
                    fop_row['mention_features'] = ",".join(mention_features)
                
                    result.write("POS_fops:"+str(POS_fops)+"\n") ;pattern_text.write("POS_fops:"+str(POS_fops)+"\n")
                    result.write("DEP_fops:"+str(DEP_fops)+"\n") ;pattern_text.write("DEP_fops:"+str(DEP_fops)+"\n")
#                     f.write("PMI_fops:"+str(cand_pairs_score)+"\n")
#                     print(cand_pairs_score)
                    result.write("************************************************\n") 
                    if "not" in token_set: continue              
                    if overall > 3 and sentiment.polarity < 0: continue
                    if overall < 3 and sentiment.polarity > 0: continue
                    if (len(POS_fops) == 0) and (len(DEP_fops) == 0): continue
                        
#                     fop_list = set(PMI_fops + POS_fops + DEP_fops)
                    fop_list = set(POS_fops + DEP_fops)
                    fop_prob_list = []
                    
                    for fop in fop_list:
                        p1,p2,p3 = 0,0,0; y1,y2,y3 = 0.6,0.2,0.2;
                        feat,opinion = fop
#                         if fop in PMI_fops: 
#                             p1 = pmi(feat,opinion)
                        if fop in POS_fops: p2 = 1
                        if fop in DEP_fops: p3 = 1
#                         p = y1*p1 + y2*p2 + y3*p3
                        p = y2*p2 + y3*p3
                        fop_prob_list.append((fop,p))
                    result.write("fop_prob_list:" + str(fop_prob_list)+ "\n")   
                    i +=1 
                    result.write("\n");pattern_text.write("\n")
                    # ------------------------------------------------------
#                     training_data.write("%s"%(rev_sent) + "|") 
#                     training_data.write("%s|%s|"%(sentiment.polarity,sentiment.subjectivity))
#                     if len(token_set & total_keywords) == 0: continue                    
                    
                    fop_str = ""
                    for idx,fop_score in enumerate(fop_prob_list):
                        fop,score = fop_score
                        feat,o = fop                        
                        if idx == len(fop_prob_list) - 1:
                            fop_str += "%s %s"%(feat,o)
                        else:     
                            fop_str += "%s %s,"%(feat,o)
                    fop_row["fops"] = fop_str 
                    keyword_list.append(fop_str)
                    fop_row['review_ID'] = review_ID
                    fop_row['summary'] = lemm_sent_process2(summary, remove_stopwords=False, summary=False, mode="spacy", withdot=False)[0]
                    
                    if abs(sentiment.polarity) <= 0.5: continue
                    if abs(sentiment.subjectivity) < 0.4: continue
                    if len(mention_features) == 0: continue
                    topic_train_df[i] = fop_row ;i +=1       
                
                    
                total_keyword = " ".join(keyword_list)
                rev_summ_row['review_ID'] = review_ID
                rev_summ_row['big_categories'] = big_categories
                rev_summ_row['small_categories'] = small_categories
                rev_summ_row['review'] = review
                rev_summ_row['summary'] = summary
                rev_summ_row['big_categories'] = big_categories
                rev_summ_row['small_categories'] = small_categories
                rev_summ_row['lemm_review'] = lemm_review
                rev_summ_row['lemm_summary'] = lemm_summary
                rev_summ_row['lemm_review_len'] = lemm_review_len
                rev_summ_row['lemm_summary_len'] = lemm_summary_len
                rev_summ_row['overall'] = overall
                rev_summ_row['total_keyword'] = total_keyword
                key_train_df[j] = rev_summ_row;
                j += 1    
                if j > 50: break
#                 f.write("---------------------------------------------------\n")
            
#             break  
            
print("finished...")
# training_data.close()
topic_train_df = pd.DataFrame.from_dict(topic_train_df, orient='index')
key_train_df = pd.DataFrame.from_dict(key_train_df, orient='index')

Product : B01GW45SB8 , Processing ID 1471132800  :   7%|▋         | 190/2564 [00:30<06:18,  6.28it/s]

finished...


# topic-to-eaasy DataSet 建立

In [16]:
csv_path = "Train/%s_%s_topic.xlsx"%(category1,category2)
# df.to_csv(csv_path) #默认dt是DataFrame的一个实例，参数解释如下
topic_train_df.to_excel(csv_path, encoding='utf8')
print(csv_path + " Write finished")
print(" Write finished")
topic_train_df.head()
len(topic_train_df)

Train/Cell Phones & Accessories_Cell Phones_topic.xlsx Write finished
 Write finished


47

# Key word Attention DataSet 建立

In [17]:
csv_path = "Train/%s_%s_key.xlsx"%(category1,category2)
# df.to_csv(csv_path) #默认dt是DataFrame的一个实例，参数解释如下
key_train_df.to_excel(csv_path, encoding='utf8')
print(csv_path + " Write finished")
print(" Write finished")
len(key_train_df)

Train/Cell Phones & Accessories_Cell Phones_key.xlsx Write finished
 Write finished


145

In [ ]:
key_train_df.head()